In [ ]:
import sys
import os
import import_ipynb
import numpy as np
import random
from termcolor import colored
import gym
from gym.spaces import Discrete, Box, MultiDiscrete
import numpy as np
from stable_baselines import PPO2, A2C, DQN
#from stable_baselines.common.policies import MlpPolicy
from stable_baselines.deepq.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.common.env_checker import check_env
import Cube

class RubiksEnv(gym.Env):
    def __init__(self, train = True):
        self.action_space = Discrete(12)
        self.observation_space = MultiDiscrete([2]*162 + [10])
        self.reset_number = 0
        self.train = train
    def _get_move(self, action):
        if action == 0:
            move = 'R'
        elif action == 1:
            move = 'R\''
        elif action == 2:
            move = 'L'
        elif action == 3:
            move = 'L\''
        elif action == 4:
            move = 'U'
        elif action == 5:
            move = 'U\''
        elif action == 6:
            move = 'D'
        elif action == 7:
            move = 'D\''
        elif action == 8:
            move = 'F'
        elif action == 9:
            move = 'F\''
        elif action == 10:
            move = 'B'
        elif action == 11:
            move = 'B\''
        return move
    
    def step(self, action):
        if not self.cube.isdone():
            move = self._get_move(action)
            self.cube.do(move)
            self.actions.append(move)
        self.move_number +=1
        
        if self.cube.isdone():
            reward = 100
        else:
            reward = -1
        
        done = self.cube.isdone()
        if self.train == True:
            if self.move_number >= max(60/(0.5*np.log(1+self.reset_number)),30*self.shuffle_number//5):
            #if self.move_number >= 80:
                reward = -10
                done = True
        else:
            if self.move_number >=40:
                reward = -10
                done = True
        
        #obs = np.append(self.cube.get_array(),(1 + self.reset_number//250)%10)
        obs = np.append(self.cube.get_array(),self.shuffle_number)
        
        return obs, reward, done, {}
        
    def reset(self):
        self.cube = Cube()
        self.cube.reset()
        self.reset_number += 1
        if self.reset_number%2000 == 0:
            random.seed(self.reset_number)
        if self.train == True and self.reset_number<=24000:
            self.shuffle_number = random.randint(min(0+self.reset_number//2000,6),min(4 + self.reset_number//2800,15)) 
        elif self.train == True:
            self.shuffle_number = random.randint(0,15)
        else:
            self.shuffle_number = random.randint(1,15)
        self.actions = []
        self.move_number = 0
        #self.cube.shuffle((1 + self.reset_number//250)%10, seed = random.randint(5,15))
        self.cube.shuffle(self.shuffle_number, seed = random.randint(5,15))
        #return np.append(self.cube.get_array(),(1 + self.reset_number//250)%10)
        return np.append(self.cube.get_array(),self.shuffle_number)
    
    def render(self, mode='human', close=False):
        self.cube.render()
        
    def is_done(self):
        return self.cube.isdone()